# Onboard a Credit Approval Model to Evaluate Fairness

In this notebook, we present the steps for onboarding a model to evaluate model fairness.  

Fiddler is the pioneer in enterprise Model Performance Management (MPM), offering a unified platform that enables Data Science, MLOps, Risk, Compliance, Analytics, and LOB teams to **monitor, explain, analyze, and improve ML deployments at enterprise scale**. 
Obtain contextual insights at any stage of the ML lifecycle, improve predictions, increase transparency and fairness, and optimize business revenue.

---

You can experience Fiddler's Fairness Offering ***in minutes*** by following these four quick steps:

1. Connect to Fiddler
2. Define Your Model Specifications
3. Add Your Model
4. Upload The Model Package 
5. Get Fairness insights


# 0. Imports

In [ ]:
!pip install -q fiddler-client

import fiddler as fdl
import pandas as pd
import yaml
import datetime
import time
from IPython.display import clear_output

print(f"Running Fiddler client version {fdl.__version__}")

# 1. Connect to Fiddler

Before you can add information about your model with Fiddler, you'll need to connect using our Python client.

---

**We need a few pieces of information to get started.**
1. The URL you're using to connect to Fiddler
3. Your authorization token

The latter two of these can be found by pointing your browser to your Fiddler URL and navigating to the **Settings** page.

In [24]:
URL = ''  # Make sure to include the full URL (including https://).
TOKEN = ''

Now just run the following code block to connect the client to your Fiddler environment.

In [25]:
fdl.init(
    url=URL,
    token=TOKEN)

Once you connect, you can create a new project by specifying a unique project ID in the client's [create_project](https://docs.fiddler.ai/reference/clientcreate_project) function.

In [27]:
PROJECT_NAME = 'credit_approval'

project = fdl.Project(
    name=PROJECT_NAME)

project.create()

# 2. Define Your Model 

In this example, we'll be considering the case where we're a bank and we have **a model that predicts credit approval worthiness**.
  
In order to get insights into the model's performance, **Fiddler needs a small  sample of data that can serve as a baseline** for making comparisons with data in production.


---


*For more information on how to design a sample dataset, [click here](https://docs.fiddler.ai/docs/designing-a-baseline-dataset).*

In [28]:
PATH_TO_SAMPLE_CSV = 'https://media.githubusercontent.com/media/fiddler-labs/fiddler-examples/main/quickstart/data/intersectionally_unfair_baseline.csv'

sample_df = pd.read_csv(PATH_TO_SAMPLE_CSV)
sample_df

,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,CNT_FAM_MEMBERS,gender,race,income,paid_off,#_of_pastdues,no_loan,target,Approve_probability_of_credit_request
0,1,1,2,1,0,0,32.890411,-12.443836,2.0,M,Other,49306.571969,0,3,1,0,0.024598
1,1,1,2,2,0,0,58.832877,-3.106849,2.0,F,Caucasian,139386.670908,9,10,0,0,0.200638
2,0,1,2,2,1,0,52.356164,-8.358904,1.0,M,Caucasian,144281.758418,0,0,22,1,0.826015
3,0,1,0,1,1,0,61.545205,1000.665753,1.0,F,Asian,158338.663878,0,0,15,1,0.808336
4,1,1,2,1,0,0,46.224658,-2.106849,2.0,F,Caucasian,134150.633849,0,0,60,1,0.986340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45980,1,1,2,2,0,0,38.334247,-2.178082,2.0,M,Asian,131966.029281,6,12,0,0,0.041951
45981,1,1,2,2,0,0,30.865753,-7.019178,4.0,M,Asian,129770.545151,0,0,18,1,0.685311
45982,0,1,2,2,1,0,36.512329,-9.936986,1.0,M,Caucasian,145865.051770,1,12,0,0,0.016415
45983,0,1,2,2,0,0,40.002740,-1.093151,4.0,F,Caucasian,120365.414155,0,2,0,0,0.180347


### 2.a Define Model Specification
In order to add your model to Fiddler, simply create a ModelSpec object with information about what each column of your data sample should used for.

Fiddler supports four column types:
1. **Inputs**
2. **Outputs** (Model predictions)
3. **Targets** (Ground truth values)
4. **Metadata**

In [31]:
model_spec = fdl.ModelSpec(
    inputs=['FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       'CNT_FAM_MEMBERS', 'income', 'paid_off', '#_of_pastdues', 'no_loan'],
    outputs=['Approve_probability_of_credit_request'],
    targets=['target'],
    metadata=['gender','race'])

### 2.b Define Model Task

Fiddler supports a variety of model tasks. In this case, we're adding a binary classification model.

For this, we'll create a ModelTask object and an additional ModelTaskParams object to specify the ordering of our positive and negative labels.

For a detailed breakdown of all supported model tasks, click here.

In [32]:
model_task = fdl.ModelTask.BINARY_CLASSIFICATION

task_params = fdl.ModelTaskParams(target_class_order=[0, 1])

## 3. Add your model

Create a Model object and publish it to Fiddler, passing in:
1. Your data sample
2. Your ModelSpec object
3. Your ModelTask and ModelTaskParams objects

In [33]:
MODEL_NAME = 'intersectionally_unfair'

model = fdl.Model.from_data(
    name=MODEL_NAME,
    project_id=fdl.Project.from_name(PROJECT_NAME).id,
    source=sample_df,
    spec=model_spec,
    task=model_task,
    task_params=task_params)

model.create()

## 4. Upload your model package

Now it's time to upload your model package to Fiddler.  To complete this step, we need to ensure we have 2 assets in a directory.  It doesn't matter what this directory is called, but for this example we will call it **/model**.

In [34]:
import os
os.makedirs("model")

***Your model package directory will need to contain:***
1. A **package.py** file which explains to Fiddler how to invoke your model's prediction endpoint
2. And the **model artifact** itself
3. (Optional) A **requirements.txt** specifying which python libraries need by package.py.  This example doesn't require any additional libraries to be installed so a requirements.txt file is not needed here.


### 4.a Create the **package.py** file

The contents of the cell below will be written into our ***package.py*** file.  This is the step that will be most unique based on model type, framework and use case.  The model's ***package.py*** file also allows for preprocessing transformations and other processing before the model's prediction endpoint is called.  For more information about how to create the ***package.py*** file for a variety of model tasks and frameworks, please reference the [Uploading a Model Artifact](https://docs.fiddler.ai/docs/uploading-a-model-artifact#packagepy-script) section of the Fiddler product documentation.

In [35]:
%%writefile model/package.py

import pickle
from pathlib import Path
import pandas as pd

PACKAGE_PATH = Path(__file__).parent

class SklearnModelPackage:

    def __init__(self):
        self.is_classifier = True
        self.is_multiclass = False
        self.output_columns = ['Approve_probability_of_credit_request']
        with open(PACKAGE_PATH / 'model_unfair.pkl', 'rb') as infile:
            self.model = pickle.load(infile)

    def predict(self, input_df):
        if self.is_classifier:
            if self.is_multiclass:
                predict_fn = self.model.predict_proba
            else:
                def predict_fn(x):
                    return self.model.predict_proba(x)[:, 1]
        else:
            predict_fn = self.model.predict
        return pd.DataFrame(predict_fn(input_df), columns=self.output_columns)

def get_model():
    return SklearnModelPackage()

Writing model/package.py


### 4.b  Ensure your model's artifact is in the **/model** directory

Make sure your model artifact (*e.g. the model_unfair.pkl file*) is also present in the model package directory.  The following cell will move this model's pkl file into our */model* directory.

In [36]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/models/model_unfair.pkl", "model/model_unfair.pkl")

('model/model_unfair.pkl', <http.client.HTTPMessage at 0x117d56fe0>)

### 4.c Upload Sample Dataset 

In [37]:
DATASET_NAME = 'baseline' 
#Publishing Pre-prod dataset
model.publish(
  source=sample_df, # we are using the same sample dataset we used earlier in model onboarding
  environment=fdl.EnvType.PRE_PRODUCTION, #environemnt parameter is used to designate this dataset as a pre-production data
  dataset_name=DATASET_NAME)

### 4.d Define Model Parameters and Upload Model Files

In [38]:
MODEL_DIR = 'model/'
DEPLOYMENT_PARAMS = {'memory': 1024, 'cpu': 1000}
model.add_artifact(
  model_dir=MODEL_DIR,
  deployment_params=fdl.DeploymentParams(**DEPLOYMENT_PARAMS)
)

Within your Fiddler environment's UI, you should now be able to see the newly created model.

# 5. Get Fairness insights

**You're all done!**
  
Now just head to your Fiddler environment's UI and explore the model's fairness metrics by navigating to the model selecting the fairness tab on the top right.


<table>
    <tr>
        <td>
            <img src="https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/images/Fairness.png"/>
        </td>
    </tr>
</table>



---


**Questions?**  
  
Check out [our docs](https://docs.fiddler.ai/) for a more detailed explanation of what Fiddler has to offer.

If you're still looking for answers, fill out a ticket on [our support page](https://fiddlerlabs.zendesk.com/) and we'll get back to you shortly.